# 04 - Machine Learning: Vorhersage des BIP

Dieses Notebook dokumentiert die Entscheidungen für das Machine Learning Modell.

**Der Code befindet sich in:** `src/features/train_model.py`

**Ausführung:** `python src/main.py`

---

## Ziel

Wir wollen wissen: **Welche Bildungsfaktoren sind am wichtigsten für die Vorhersage des BIP pro Kopf?**

Die Korrelationsanalyse in Notebook 03 hat uns paarweise Zusammenhänge gezeigt. Ein Machine Learning Modell kann uns zeigen, welche Features in Kombination am meisten zur Vorhersage beitragen.

---

## Entscheidung 1: Modellwahl

### Random Forest Regressor

Wir verwenden einen **Random Forest**, weil:

1. **Feature Importance:** Random Forest liefert direkt die Wichtigkeit jedes Features
2. **Nicht-lineare Zusammenhänge:** Kann komplexere Beziehungen erfassen als lineare Regression
3. **Robust:** Weniger anfällig für Overfitting als einzelne Decision Trees

Zum Vergleich trainieren wir auch eine **Lineare Regression** als Baseline.

---

## Entscheidung 2: Features und Target

### Target (y)

**BIP pro Kopf, PPP** (`NY.GDP.PCAP.PP.CD`)

PPP (Purchasing Power Parity) ist besser für Ländervergleiche geeignet, weil es die unterschiedlichen Preisniveaus berücksichtigt.

### Features (X)

| Feature | Code | Beschreibung |
|---------|------|-------------|
| Primärbildung | SE.PRM.ENRR | Einschulungsrate Grundschule (% brutto) |
| Sekundarbildung | SE.SEC.ENRR | Einschulungsrate Sekundarstufe (% brutto) |
| Tertiärbildung | SE.TER.ENRR | Einschulungsrate Hochschule (% brutto) |
| Bildungsausgaben | SE.XPD.TOTL.GD.ZS | Bildungsausgaben (% des BIP) |

Diese Features decken verschiedene Bildungsaspekte ab:
- Verschiedene Bildungsstufen (Primär, Sekundär, Tertiär)
- Investitionen in Bildung (Ausgaben)

---

## Entscheidung 3: Umgang mit fehlenden Werten

### Problem

Nicht alle Länder haben für alle Jahre vollständige Daten. Sklearn-Modelle können nicht mit NaN-Werten umgehen.

### Lösung: SimpleImputer

Wir verwenden `SimpleImputer(strategy="mean")`.

Fehlende Werte werden durch den Mittelwert der jeweiligen Spalte ersetzt. Das ist eine einfache, aber effektive Strategie für unseren Datensatz.

---

## Entscheidung 4: Train/Test Split

### Aufteilung

- **80% Trainingsdaten** – zum Trainieren des Modells
- **20% Testdaten** – zur Evaluation (das Modell hat diese Daten nie gesehen)

### Warum?

Wenn wir das Modell auf denselben Daten evaluieren, auf denen es trainiert wurde, überschätzen wir seine Leistung (Overfitting). Der Test-Score zeigt, wie gut das Modell auf neue, ungesehene Daten generalisiert.

### Code

```python
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
```

`random_state=42` sorgt für Reproduzierbarkeit.

---

## Entscheidung 5: Pipeline

Wir verwenden `make_pipeline` aus der Vorlesung (Session 14), um Imputation und Modell zu kombinieren:

```python
model = make_pipeline(
    SimpleImputer(strategy="mean"),
    RandomForestRegressor(n_estimators=100, random_state=42)
)
```

### Vorteile

1. **Kein Data Leakage:** Der Imputer wird nur auf Trainingsdaten gefittet
2. **Sauberer Code:** Ein Objekt für den gesamten Workflow
3. **Einfache Anwendung:** `model.fit()` und `model.predict()` machen alles

---

## Evaluation: R²-Score

### Was ist R²?

Der R²-Score (Bestimmtheitsmaß) misst, wie viel der Varianz im Target durch das Modell erklärt wird:

- **R² = 1.0** → Perfekte Vorhersage
- **R² = 0.0** → Modell nicht besser als der Mittelwert
- **R² < 0.0** → Modell schlechter als der Mittelwert

### Interpretation

- **Train R²:** Wie gut passt das Modell zu den Trainingsdaten?
- **Test R²:** Wie gut generalisiert das Modell auf neue Daten?

Ein großer Unterschied (Train >> Test) deutet auf Overfitting hin.

---

## Feature Importance

### Was ist Feature Importance?

Random Forest berechnet, wie viel jedes Feature zur Vorhersage beiträgt. Features mit hoher Importance sind wichtiger für die Vorhersage.

### Zugriff

```python
rf_model.feature_importances_
```

### Wichtig

Feature Importance zeigt **Vorhersagekraft**, nicht **Kausalität**! Ein Feature kann wichtig für die Vorhersage sein, ohne das BIP direkt zu verursachen.

---

## Output

Das Training (`python src/main.py`) erzeugt:

| Datei | Beschreibung |
|-------|-------------|
| `models/random_forest.joblib` | Das trainierte Modell |
| `models/feature_importance.csv` | Feature Importance Tabelle |

Das Modell kann später mit `joblib.load()` geladen werden, um Vorhersagen zu machen.

---

## Erwartete Ergebnisse

Basierend auf den Korrelationsanalysen aus Notebook 03 erwarten wir:

1. **Tertiärbildung** sollte die höchste Feature Importance haben (ρ = 0.77 mit BIP)
2. **Sekundarbildung** sollte ebenfalls wichtig sein (ρ = 0.80 mit BIP)
3. **Primärbildung** sollte wenig beitragen (ρ = 0.06 mit BIP)
4. **Bildungsausgaben** könnten überraschen – trotz schwacher Korrelation (ρ = 0.26) könnte das Feature in Kombination mit anderen wichtig sein

---

## Einschränkungen

1. **Nur Bildungsindikatoren:** Andere wichtige Faktoren (Politik, Geographie, Geschichte, Ressourcen) fehlen
2. **Keine Kausalität:** Das Modell zeigt Zusammenhänge, keine Ursache-Wirkung
3. **Imputation:** Fehlende Werte wurden durch Mittelwerte ersetzt – das kann die Ergebnisse verzerren
4. **Panel-Daten:** Wir behandeln jede Land-Jahr-Kombination als unabhängige Beobachtung, obwohl Länder über Zeit korreliert sind